- declarations
- get table of conceptual entities in target model, along with the groups they are in
- put them in a json dict
- get table of links between conceptual entities
- put them in a json dict
- spin the json dict into the jinja template for yed

In [1]:
import json
import os
import sqlite3

from pathlib import Path
from jinja2 import Template, Environment
from datetime import datetime

conn = sqlite3.connect('app_db.sqlite')
cur = conn.cursor()

template_path = "export_templates/yed_live/"
template_filename = "conceptual_er_w_types_and_areas.tem"
template = Path(os.path.join(template_path, template_filename)).read_text()

time_string = datetime.now().strftime('%Y%m%d%H%M%S')
output_path = "export_output_files/yed_live/"
output_filename = "conceptual_er_w_types_and_areas_" + time_string + ".graphml"


In [2]:
# generate the JSON to load into the template
# this could be a separate script generating a JSON output
# but that seems redundant at the moment

# First, get the types to make the group nodes

data = {}

sql_query = """
-- Get ConceptualEntityTypes w/ BusinessArea
SELECT
    h_ce.ConceptualEntityKeyPhrase
  , s_ces.Name AS ConceptualEntityName
  , s_ces.Description AS ConceptualEntityDescription
  , s_ces.Definition
  , h_ba.BusinessAreaKeyPhrase
  , s_ba.BusinessAreaName
  , s_ba.Description AS BusinessAreaDescription
FROM
  rv_h_ConceptualEntity AS h_ce
  INNER JOIN rv_s_ConceptualEntity_iServer AS s_ces ON (
    h_ce.ConceptualEntityHashKey = s_ces.ConceptualEntityHashKey
    AND s_ces.LoadDate = (
      SELECT MAX(z.LoadDate)
      FROM rv_s_ConceptualEntity_iServer AS z
      WHERE z.ConceptualEntityHashKey = s_ces.ConceptualEntityHashKey
    )
  )
  -- only Concepts that are in Business Areas
  INNER JOIN rv_l_BusinessAreaConceptualEntity AS l_bace ON (
    h_ce.ConceptualEntityHashKey = l_bace.ConceptualEntityHashKey
  )
  INNER JOIN rv_h_BusinessArea AS h_ba ON (
    l_bace.BusinessAreaHashKey = h_ba.BusinessAreaHashKey
  )
  INNER JOIN rv_s_BusinessArea AS s_ba ON (
    h_ba.BusinessAreaHashKey = s_ba.BusinessAreaHashKey
    AND s_ba.LoadDate = (
      SELECT MAX(z.LoadDate)
      FROM rv_s_BusinessArea AS z
      WHERE z.BusinessAreaHashKey = s_ba.BusinessAreaHashKey
    )
  )
WHERE
  s_ces.isDeleted = 0
  AND h_ce.ConceptualEntityHashKey IN (
    SELECT DISTINCT
      ConceptualEntity_TypeOfHashKey
    FROM
      rv_l_ConceptualEntityConceptualEntity_TypeOf
  )
;
""";
cur.execute(sql_query)

data['conceptual_entity_type_groups'] = {}
for row in cur:
  #print(row)
  conceptual_entity_type_group = {
    "CEkey":row[0]
    , "name":row[1]
    , "description":row[2]
    , "definition":row[3]
    , "BAkey":row[4]
    , "BAname":row[5]
    , "BAdescription":row[6]
    , "entitys":[]
  }
  data['conceptual_entity_type_groups'][row[0]] = conceptual_entity_type_group

#print(data)

In [3]:
# now get the entities that aren't the types

sql_query = """
-- Get ConceptualEntity w/ BusinessArea and Type
SELECT
    h_ce.ConceptualEntityKeyPhrase
  , s_ces.Name AS ConceptualEntityName
  , s_ces.Description AS ConceptualEntityDescription
  , s_ces.Definition
  , h_ba.BusinessAreaKeyPhrase
  , s_ba.BusinessAreaName
  , s_ba.Description AS BusinessAreaDescription
  , h_cet.ConceptualEntityKeyPhrase AS ConceptualEntity_TypeOfKeyPhrase
  , s_cets.Name AS ConceptualEntity_TypeOfName
FROM
  rv_h_ConceptualEntity AS h_ce
  INNER JOIN rv_s_ConceptualEntity_iServer AS s_ces ON (
    h_ce.ConceptualEntityHashKey = s_ces.ConceptualEntityHashKey
    AND s_ces.LoadDate = (
      SELECT MAX(z.LoadDate)
      FROM rv_s_ConceptualEntity_iServer AS z
      WHERE z.ConceptualEntityHashKey = s_ces.ConceptualEntityHashKey
    )
  )
  -- only Concepts that are in Business Areas
  INNER JOIN rv_l_BusinessAreaConceptualEntity AS l_bace ON (
    h_ce.ConceptualEntityHashKey = l_bace.ConceptualEntityHashKey
  )
  INNER JOIN rv_h_BusinessArea AS h_ba ON (
    l_bace.BusinessAreaHashKey = h_ba.BusinessAreaHashKey
  )
  INNER JOIN rv_s_BusinessArea AS s_ba ON (
    h_ba.BusinessAreaHashKey = s_ba.BusinessAreaHashKey
    AND s_ba.LoadDate = (
      SELECT MAX(z.LoadDate)
      FROM rv_s_BusinessArea AS z
      WHERE z.BusinessAreaHashKey = s_ba.BusinessAreaHashKey
    )
  )
  -- maybe Concepts have Types
  LEFT JOIN rv_l_ConceptualEntityConceptualEntity_TypeOf AS l_cecet ON (
    h_ce.ConceptualEntityHashKey = l_cecet.ConceptualEntityHashKey
  )
  LEFT JOIN rv_h_ConceptualEntity AS h_cet ON (
    l_cecet.ConceptualEntity_TypeOfHashKey = h_cet.ConceptualEntityHashKey
  )
  LEFT JOIN rv_s_ConceptualEntity_iServer AS s_cets ON (
    h_cet.ConceptualEntityHashKey = s_cets.ConceptualEntityHashKey
    AND s_cets.LoadDate = (
      SELECT MAX(z.LoadDate)
      FROM rv_s_ConceptualEntity_iServer AS z
      WHERE z.ConceptualEntityHashKey = s_cets.ConceptualEntityHashKey
    )
  )
WHERE
  s_ces.isDeleted = 0
  AND h_ce.ConceptualEntityHashKey NOT IN (
    SELECT DISTINCT
      ConceptualEntity_TypeOfHashKey
    FROM
      rv_l_ConceptualEntityConceptualEntity_TypeOf
  )
;
""";
cur.execute(sql_query)

data['conceptual_entitys'] = []
for row in cur:
  #print(row)
  conceptual_entity = {
    "CEkey":row[0]
    , "name":row[1]
    , "description":row[2]
    , "definition":row[3]
    , "BAkey":row[4]
    , "BAname":row[5]
    , "BAdescription":row[6]
    , "TYkey":row[7]
    , "TYname":row[8]
  }
  if (row[7] is None):
    data['conceptual_entitys'].append(conceptual_entity)
  else:
    data['conceptual_entity_type_groups'][row[7]]['entitys'].append(conceptual_entity)

#print(data)

In [4]:
# drop the JSON into the Template

j2_template = Template(template, autoescape=True)

rendered_template_string = j2_template.render(data)
with open(os.path.join(output_path, output_filename), "w") as text_file:
  text_file.write(rendered_template_string)